# Домашнее задание к лекции "Продвинутый pandas"

Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [9]:
import pandas as pd
import numpy as np
data = pd.read_csv("\\Users/tatyana.frolova/Downloads/Python_13_join/ml-latest-small/ratings.csv")


#ratings_pivot = data.pivot_table(data ,index='userId', aggfunc={'userId':np.size, 'timestamp':(np.max,np.min)})
#ratings_pivot = data.group(data ,index='userId', aggfunc={'userId':np.size})

t_minmax = data.groupby('userId').agg(['min', 'max'])['timestamp']
t_count = data.groupby('userId').agg(['count'])['movieId']




tab_grouped = data.groupby('userId').agg({'timestamp': ['min', 'max'], 'movieId': 'count'})
sample = tab_grouped.loc[tab_grouped['movieId']['count'] > 100]
print('******************************************')
print(sample.head(10))
print('******************************************')
print('Среднее время жизни: ' + str( (sample['timestamp']['max'].sum()-sample['timestamp']['min'].sum())/len(sample)))





******************************************
         timestamp             movieId
               min         max   count
userId                                
4        949778714   949982274     204
8       1154389340  1154474527     116
15       997937239  1469330735    1700
17      1127468587  1127476640     363
19       855190091   855195373     423
21       853157476   854522908     162
22      1131661890  1131753381     220
23      1148386124  1166728253     726
26      1351544316  1371811577     172
30       944943070  1060795346    1011
******************************************
Среднее время жизни: 40080507.4496124


Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [6]:
import pandas as pd
import numpy as np


rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118,200],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4','bjfevferjui']
    }
)





#------------------------------------------------
# таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
rzd_auto_air = rzd.merge(auto, left_on='client_id', right_on='client_id', how='outer').merge(air, left_on='client_id', right_on='client_id', how='outer')
print('*******************')
print('Таблица без адреса')
print('*******************')
print(rzd_auto_air)
#аналогичную таблицу по типам выручки с указанием адреса клиента
result_address = rzd_auto_air.merge(client_base, left_on='client_id', right_on='client_id', how='left')
print('*******************')
print('Таблица с адресом')
print('*******************')
print(result_address)
#------------------------------------------------
#------------------------------------------------
# ИЛИ
rzd = rzd.set_index('client_id')
auto = auto.set_index('client_id')
air = air.set_index('client_id')
client_base = client_base.set_index('client_id')
result = rzd.join(auto, how='outer').join(air, how='outer')
print('*******************')
print('Таблица без адреса')
print('*******************')
print(result)
full_result = result.join(client_base, how='left')
print('*******************')
print('Таблица с адресом')
print('*******************')
print(full_result)



*******************
Таблица без адреса
*******************
   client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           NaN          NaN
1        112       2810.0           NaN          NaN
2        113      10283.0       57483.0          NaN
3        114       5774.0          83.0          NaN
4        115        981.0         912.0         81.0
5        116          NaN        4834.0          4.0
6        117          NaN          98.0         13.0
7        118          NaN           NaN        173.0
*******************
Таблица с адресом
*******************
   client_id  rzd_revenue  auto_revenue  air_revenue          address
0        111       1093.0           NaN          NaN  Комсомольская 4
1        112       2810.0           NaN          NaN   Энтузиастов 8а
2        113      10283.0       57483.0          NaN  Левобережная 1а
3        114       5774.0          83.0          NaN          Мира 14
4        115        981.0         912.0         81.0    

Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [5]:
import pandas as pd

visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad'],
        'time': ['1154399340', '1154382340', '1154381230', '1154345640', '1100000000'],
    }
)

visits = visits[['user_id', 'source']]
visits

purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
        'purtime': ['1154399340', '1154382340', '1154381230', '1154345640', '1100000000'],
        'time': ['1154399340', '1154382340', '1154381230', '1154345640', '1100000000'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases


coordinates = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'time': ['1154399340', '1154382340', '1154381230', '1154345640', '1100000000'],
        'latitude': ['56,3287', '100,3287', '60,3287', '20,3287', '80,3287'],
        'longitude': ['44,002', '20,002', '50,002', '40,002', '70,002']
    }
)

coordinates = coordinates[['user_id', 'time']]
coordinates

,user_id,time
0,11,1154399340
1,22,1154382340
2,55,1154381230
3,11,1154345640
4,99,1100000000
